In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
df = pd.read_csv("Scraping Profesi Data (Jobstreet).csv")

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
df.columns

In [ ]:
df.drop(["Kualifikasi", "Spesialisasi Pekerjaan", "No. Registrasi", "Waktu Proses Lamaran", "Tunjangan dan Lain-lain", "Lokasi", "No. EA"], axis = 1, inplace = True)

In [ ]:
df.rename(columns = {"Tingkat Pekerjaan": "tingkat_job", 
                     "Pengalaman Kerja": "lama_pengalaman", "Jenis Pekerjaan": "jenis_job", 
                     "Ukuran Perusahaan": "ukuran_company", "Industri": "industri"}, inplace=True)

In [ ]:
df.head(3)

# Preprocessing 1

### Ganti Job Name dan Lokasi

In [ ]:
data = df.copy()
data.head(3)

In [ ]:
data.info()

In [ ]:
# Cek data yang duplikat
data[data.duplicated(keep = False)] 

# Data Analyst/ Data Scientist -> Ini dihapus saja agar tidak bingung mau dimasukkan ke "data analyst" atau "data scientist"

In [ ]:
data.drop(data.index[data["job_name"] == "DATA ANALYST / DATA SCIENTIST"], inplace = True)
data.drop_duplicates(keep = "first", inplace = True, ignore_index = True)
data.info() # Ada 4 yang terhapus

In [ ]:
data[data.duplicated(keep = False)] # Sudah tidak ada yang duplikat

In [ ]:
data.index # Index-nya sudah urut

In [ ]:
data["lokasi"].unique()

In [ ]:
# PROSESNYA DIGABUNG JADI SATU

def ganti_nama_dan_lokasi(data, list_kolom):
    data2 = data.copy()
    
    # Looping untuk lower case isi dari masing-masing kolom
    for kolom in list_kolom:
        data2[kolom] = data2[kolom].str.lower()

    # Looping untuk ganti isi di kolom "job_name" dan "lokasi"
    for i in range(0, len(data2)):
        # Untuk kolom "job_name"
        # ["data scientist", "data analyst", "data engineer", "database manager", "data architect", "database administrator"]
        if "data scientist" in data2["job_name"][i]:
            data2["job_name"][i] = "data scientist"
        elif "data analyst" in data2["job_name"][i]:
            data2["job_name"][i] = "data analyst"
        elif "data engineer" in data2["job_name"][i]:
            data2["job_name"][i] = "data engineer"
        elif "database manager" in data2["job_name"][i]:
            data2["job_name"][i] = "database manager"
        elif "data architect" in data2["job_name"][i]:
            data2["job_name"][i] = "data architect"
        elif "database administrator" in data2["job_name"][i]:
            data2["job_name"][i] = "database administrator"

        # Untuk kolom "lokasi"        
        data2["lokasi"].replace(["jakarta raya", "jakarta selatan", "jakarta barat", "jakarta pusat", 
                                 "jakarta timur", "jakarta utara"], "dki jakarta", inplace = True)
        data2["lokasi"].replace(["tangerang", "serang"], "banten", inplace = True)
        data2["lokasi"].replace("yogyakarta", "diy", inplace = True)
        data2["lokasi"].replace("badung", "bali", inplace = True)
        data2["lokasi"].replace("paser", "kalimantan timur", inplace = True)
        data2["lokasi"].replace(["bandung", "bekasi", "cikarang"], "jawa barat", inplace = True)
        data2["lokasi"].replace(["surabaya", "sidoarjo"], "jawa timur", inplace = True)
        data2["lokasi"].replace("medan", "sumatera utara", inplace = True)
    
    return data2

In [ ]:
list_kolom = list(data.columns)

data_oke = ganti_nama_dan_lokasi(data, list_kolom)
data_oke.head()

In [ ]:
data_oke[data_oke["lokasi"] == "indonesia"]

In [ ]:
data_oke[data_oke["lokasi"] == "beberapa lokasi kerja"]

In [ ]:
# Yang lokasinya di "indonesia" akan diganti
data_oke.iloc[115].replace("indonesia", "jawa barat", inplace = True)
data_oke.iloc[164].replace("indonesia", "jawa barat", inplace = True)
data_oke.iloc[167].replace("indonesia", "dki jakarta", inplace = True)

# Yang lokasinya di "beberapa lokasi kerja" akan diganti
data_oke.iloc[132].replace("beberapa lokasi kerja", "dki jakarta", inplace = True)
data_oke.iloc[159].replace("beberapa lokasi kerja", "dki jakarta", inplace = True)
data_oke.iloc[183].replace("beberapa lokasi kerja", "jawa barat", inplace = True)

In [ ]:
data_oke["lokasi"].unique()

In [ ]:
#data_oke.to_csv("Hasil Preprocessing 1 (Ganti Job Name dan Lokasi) Data Jobstreet.csv", index = False)

# Preprocessing 2

### Penyamaan/Standarisasi Kolom "jenis_job"

In [ ]:
data_oke = pd.read_csv("Hasil Preprocessing 1 (Ganti Job Name dan Lokasi) Data Jobstreet.csv")
data_oke.tail()

In [ ]:
data_oke["jenis_job"].unique()

In [ ]:
data_oke[data_oke["jenis_job"] == "penuh waktu, kontrak"]

In [ ]:
data_oke[data_oke["tingkat_job"] == "lulusan baru/pengalaman kerja kurang dari 1 tahun"]["jenis_job"]

# Dengan tingkat_job yang sama, kebanyakan adalah "penuh waktu" 
# sehingga yang isinya "penuh waktu, kontrak" akan diganti menjadi "penuh waktu" saja

In [ ]:
data_oke.iloc[85].replace("penuh waktu, kontrak", "penuh waktu", inplace = True)

In [ ]:
data_oke["jenis_job"].unique()

# Preprocessing 3

### Penyamaan/Standarisasi Kolom "tingkat_job"

In [ ]:
data_oke["tingkat_job"].unique()

In [ ]:
# 1. "lulusan baru/pengalaman kerja kurang dari 1 tahun"
data_oke[data_oke["tingkat_job"] == "lulusan baru/pengalaman kerja kurang dari 1 tahun"]

In [ ]:
for i in range(0, len(data_oke)):
    if data_oke["tingkat_job"][i] == "lulusan baru/pengalaman kerja kurang dari 1 tahun":
        if data_oke["jenis_job"][i] == "paruh waktu":
            data_oke.iloc[i].replace("lulusan baru/pengalaman kerja kurang dari 1 tahun", "magang", inplace = True)
        elif data_oke["jenis_job"][i] == "magang":
            data_oke.iloc[i].replace("lulusan baru/pengalaman kerja kurang dari 1 tahun", "magang", inplace = True)
        elif data_oke["jenis_job"][i] == "penuh waktu": # Diganti antara jadi "tingkat pemula" atau "asosiasi"
            data_oke.iloc[i].replace("lulusan baru/pengalaman kerja kurang dari 1 tahun", "tingkat pemula", inplace = True)
        elif data_oke["jenis_job"][i] == "kontrak": # Diganti antara jadi "tingkat pemula" atau "asosiasi"
            data_oke.iloc[i].replace("lulusan baru/pengalaman kerja kurang dari 1 tahun", "tingkat pemula", inplace = True)

In [ ]:
# 2. "pegawai (non-manajemen & non-supervisor)" -> mungkin masuk ke "asosiasi"
data_oke[data_oke["tingkat_job"] == "pegawai (non-manajemen & non-supervisor)"]

In [ ]:
for i in range(0, len(data_oke)):
    if data_oke["tingkat_job"][i] == "pegawai (non-manajemen & non-supervisor)":
        data_oke.iloc[i].replace("pegawai (non-manajemen & non-supervisor)", "asosiasi", inplace = True)       

In [ ]:
# 3. "supervisor/koordinator" & "manajer/asisten manajer"
# Supervisor di bawahnya manajer -> Tapi dia sama2 di posisi mid-level
data_oke[data_oke["tingkat_job"] == "supervisor/koordinator"]

In [ ]:
data_oke[data_oke["tingkat_job"] == "manajer/asisten manajer"]

In [ ]:
for i in range(0, len(data_oke)):
    if data_oke["tingkat_job"][i] == "supervisor/koordinator":
        data_oke.iloc[i].replace("supervisor/koordinator", "senior tingkat menengah", inplace = True)
    elif data_oke["tingkat_job"][i] == "manajer/asisten manajer":
        data_oke.iloc[i].replace("manajer/asisten manajer", "senior tingkat menengah", inplace = True)

In [ ]:
# 4. "tidak terspesifikasi" 
# - > Ini dianggap nan saja karena tingkat_job itu kemungkinan menentukan gaji yang berbeda 
data_oke[data_oke["tingkat_job"] == "tidak terspesifikasi"]

In [ ]:
for i in range(0, len(data_oke)):
    if data_oke["tingkat_job"][i] == "tidak terspesifikasi":
        data_oke.iloc[i].replace("tidak terspesifikasi", np.nan, inplace = True)

In [ ]:
data_oke["tingkat_job"].unique()

In [ ]:
#data_oke.to_csv("Hasil Preprocessing 2 & 3 (Penyamaan Jenis Job & Tingkat Job) Data Jobstreet.csv", index = False)

# Preprocessing 4

### Penyamaan/Standarisasi Kolom "ukuran_company"

In [ ]:
data4 = pd.read_csv("Hasil Preprocessing 2 & 3 (Penyamaan Jenis Job & Tingkat Job) Data Jobstreet.csv")
data4.head()

In [ ]:
data4["ukuran_company"].unique()

In [ ]:
# " - " dan "- " diubah menjadi "-"
data4["ukuran_company"] = data4["ukuran_company"].str.replace(" - ", "-")
data4["ukuran_company"] = data4["ukuran_company"].str.replace("- ", "-")

In [ ]:
# Disesuaikan dengan data dari Linkedin juga, sehingga ada yang diubah

for i in range(0, len(data4)):
    if data4["ukuran_company"][i] == "1001-2000 pekerja":
        data4.iloc[i] = data4.iloc[i].replace("1001-2000 pekerja", "1.001-5.000 pekerja")
    elif data4["ukuran_company"][i] == "2001-5000 pekerja":
        data4.iloc[i] = data4.iloc[i].replace("2001-5000 pekerja", "1.001-5.000 pekerja")
    
    elif data4["ukuran_company"][i] == "lebih dari 5000 pekerja":
        data4.iloc[i] = data4.iloc[i].replace("lebih dari 5000 pekerja", ">5.000 pekerja")
        
    elif data4["ukuran_company"][i] == "501-1000 pekerja":
        data4.iloc[i] = data4.iloc[i].replace("501-1000 pekerja", "501-1.000 pekerja")

In [ ]:
data4["ukuran_company"].unique()

In [ ]:
#data4.to_csv("Hasil Preprocessing 4 (Penyamaan Ukuran Company) Data Jobstreet.csv", index = False)

# Preprocessing 5

### Memperbaiki Kolom Gaji & Membuat Kolom Median Gaji

In [ ]:
data5 = pd.read_csv("Hasil Preprocessing 4 (Penyamaan Ukuran Company) Data Jobstreet.csv")
data5.head()

In [ ]:
data_oke5 = data5.copy()

# GAJI PER BULAN -> Datanya sudah gaji per bulan
for i in range(0, len(data_oke5)):
    if pd.isna(data_oke5["gaji"][i]) == True:
        pass
    else:
        data_oke5["gaji"][i] = data_oke5["gaji"][i].replace("idr ", "").replace(".", "")
        data_oke5["gaji"][i] = data_oke5["gaji"][i].split(" - ")

        if type(pd.isna(data_oke5["gaji"][i])) == type(True):
            pass
        if "sgd" in str(data_oke5["gaji"][i]):
            pass
        elif len(data_oke5["gaji"][i]) == 2: # Jika list-nya berisi 2 nilai
            list_angka = [int(angka) for angka in data_oke5["gaji"][i]]
            data_oke5["gaji"][i] = list_angka
        else:
            pass

In [ ]:
for i in range(0, len(data_oke5)):
    if type(data_oke5["gaji"][i]) == list:
        print(i)
        print(data_oke5["gaji"][i])
    
# Yang belum beres -> index 6

In [ ]:
data_oke5.iloc[6]

# Basisnya di Singapura -> Akan dihapus saja datanya

In [ ]:
data_oke5.drop(6, axis = 0, inplace = True)
data_oke5 = data_oke5.reset_index(drop = True)
len(data_oke5)

In [ ]:
# Menambah kolom baru untuk median dari gaji
import statistics

median_gaji = []
for i in range(0, len(data_oke5)):
    if type(data_oke5["gaji"][i]) != list:
        median_gaji.append(np.nan)
    else:
        median_gaji.append(round(statistics.median(data_oke5["gaji"][i])))
        
data_oke5["median_gaji"] = median_gaji

In [ ]:
data_oke5.head()

In [ ]:
#data_oke5.to_csv("Hasil Preprocessing 5 (Benerin Kolom Gaji & Bikin Kolom Median Gaji) Data Jobstreet.csv", index = False)

# Preprocessing 6

### Menyederhanakan Kolom Industri

In [ ]:
data_oke6 = pd.read_csv("Hasil Preprocessing 5 (Benerin Kolom Gaji & Bikin Kolom Median Gaji) Data Jobstreet.csv")
data_oke6.tail()

In [ ]:
data_oke6[data_oke6["industri"] == "lainnya"]

In [ ]:
data_oke6["industri"].unique()

# Misal:
# 2. Keuangan -> "perbankan/pelayanan keuangan", "asuransi"
# 10. Pendidikan -> "pendidikan"

# 1. Teknologi Informasi dan Komunikasi -> "komputer/teknik informatika (perangkat lunak)", "komputer/teknik informatika (perangkat keras)",
# -> "telekomunikasi"

# 9. Pariwisata -> "hotel/pariwisata"
# 8. Kesehatan -> "kesehatan/medis", "bioteknologi/farmasi/riset klinik"

# 7. Makanan & Minuman -> "makanan & minuman/katering/restoran"
# 4. Ekonomi -> "retail/merchandise", "umum & grosir", "produk konsumen/barang konsumen yang bergerak cepat", "manufaktur/produksi",
# 3. Konsultasi -> "konsultasi (it, ilmu pengetahuan, teknis & teknikal)", "konsultasi (bisnis & manajemen)", "manajemen/konsulting hr"
# 5. Pemasaran dan Periklanan -> "periklanan/marketing/promosi/hubungan masyarakat"
# 6. Pakaian & Mode -> "tekstil/garment"

# Ini yang udah sendiri saja: -> Punya Jobstreet
# 11. Properti -> "properti/real estate"
# 12. Pertambangan -> "pertambangan"
# 13. Elektronik -> "elektrikal & elektronik"
# 14. Agrikultur -> "agrikultural/perkebunan/peternakan unggas/perikanan"
# Sumber: https://accurate.id/bisnis-ukm/pengertian-agrikultur/
# 15. Konstruksi -> "konstruksi/bangunan/teknik"
# 16. Transportasi -> "transportasi/logistik"
# 17. Saham -> "jual beli saham/sekuritas"
# 18. Permesinan -> "industri berat/mesin/peralatan"
# Sumber: https://kemenperin.go.id/artikel/21183/Industri-Permesinan-Berperan-Kerek-Daya-Saing-Sektor-Otomotif-dan-Elektronik
# Sumber: https://www.kompasiana.com/dewayuniardi/54f33e65745513a32b6c6d07/industri-otomotif-nasional
# 19. Acara -> "pameran/manajemen acara/pikp"
# 20. BPO (Business Process Outsourcing) -> "call center/it-enabled services/bpo"
# 21. Media -> "jurnalisme", "hiburan/media" ("Media Online" punya Linkedin pindah ke "Media" aja -> Awalnya di TIK)

# Yang belum (punya Jobstreet):
# "lainnya" 
# -> Diganti jadi "Keuangan" (pt pepper services indonesia, pt cyberquote indonesia)
# Sumber: https://id.joblum.com/job/data-analytics/393544, https://jobindo.com/lowongan-kerja-perusahaan-pt-cyberquote-indonesia-9505.html

In [ ]:
industri_dict = {"perbankan/pelayanan keuangan": "keuangan", "asuransi": "keuangan", 
                 "komputer/teknik informatika (perangkat lunak)": "teknologi informasi dan komunikasi", "komputer/teknik informatika (perangkat keras)": "teknologi informasi dan komunikasi", 
                 "telekomunikasi": "teknologi informasi dan komunikasi", "hotel/pariwisata": "pariwisata", "properti/real estate": "properti",
                 "kesehatan/medis": "kesehatan", "bioteknologi/farmasi/riset klinik": "kesehatan", "tekstil/garment": "pakaian & mode",
                 "makanan & minuman/katering/restoran": "makanan & minuman", "elektrikal & elektronik": "elektronik",
                 "retail/merchandise": "ekonomi", "umum & grosir": "ekonomi", "produk konsumen/barang konsumen yang bergerak cepat": "ekonomi", "manufaktur/produksi": "ekonomi",
                 "konsultasi (it, ilmu pengetahuan, teknis & teknikal)": "konsultasi", "konsultasi (bisnis & manajemen)": "konsultasi", "manajemen/konsulting hr": "konsultasi",
                 "periklanan/marketing/promosi/hubungan masyarakat": "pemasaran & periklanan", "agrikultural/perkebunan/peternakan unggas/perikanan": "agrikultur", 
                 "konstruksi/bangunan/teknik": "konstruksi", "transportasi/logistik": "transportasi", "jual beli saham/sekuritas": "saham", 
                 "industri berat/mesin/peralatan": "permesinan", "pameran/manajemen acara/pikp": "acara", "call center/it-enabled services/bpo": "bpo", 
                 "jurnalisme": "media", "hiburan/media": "media", "lainnya": "keuangan"}

In [ ]:
data_oke6a = data_oke6.copy()
data_oke6a.head()

In [ ]:
for i in range(0, len(data_oke6a)):
    if pd.isna(data_oke6a["industri"][i]) == True:
        pass
    else:
        for awal, pengganti in industri_dict.items():
            if awal in data_oke6a["industri"][i]:
                data_oke6a["industri"][i] = data_oke6a["industri"][i].replace(awal, pengganti)

In [ ]:
data_oke6a.head()

In [ ]:
data_oke6a["industri"].nunique()

In [ ]:
#data_oke6a.to_csv("Hasil Preprocessing 6 (Menyederhanakan Kolom Industri) Data Jobstreet.csv", index = False)

# Preprocessing 7

### Memperbaiki Kolom Lama Pengalaman

In [ ]:
data_oke7 = pd.read_csv("Hasil Preprocessing 6 (Menyederhanakan Kolom Industri) Data Jobstreet.csv")
data_oke7.tail()

In [ ]:
data_oke7["lama_pengalaman"][0]

In [ ]:
data_oke7["lama_pengalaman"] = data_oke7["lama_pengalaman"].str.replace(" tahun", "")

In [ ]:
for i in range(0, len(data_oke7)):
    if pd.isna(data_oke7["lama_pengalaman"][i]) == True:
        pass
    else:
        data_oke7["lama_pengalaman"][i] = int(data_oke7["lama_pengalaman"][i])

In [ ]:
type(data_oke7["lama_pengalaman"][0])

In [ ]:
#data_oke7.to_csv("Hasil Preprocessing 7 (Memperbaiki Kolom Lama Pengalaman) Data Jobstreet.csv", index = False)

In [ ]:
##### INSYAALLOH FIX PAKAI INI (Hasil Preprocessing 7)

In [ ]:
len(data_oke7)